#### P[i] is P(X = i) (sage automatically normalizes). The second cell demonstrates that we do indeed get what we want to. 

In [217]:
U = RealDistribution('uniform', [0, 1])
#U.get_random_element()
survival = [1/2, 1/2, 5/6, 5/6, 0]
#These values for s_i give a fitness of about 1.03 (see birth_distn_experiment) when you save up to betahat = 3

In [315]:
#A squirrel is a list that looks like [birth_distn, cached_nuts]. A population of squirrels is a list
#    of such lists. Birth_distn, critically, must be a TUPLE = (b_0, b_1, . . ., b_\betahat) and not a list.

def single_squirrel_maker(birth_distn, starting_nuts = 0):
    return([birth_distn, starting_nuts])

def many_squirrel_maker(n, birth_distn, starting_nuts = 0):
    return([[birth_distn, starting_nuts] for s in range(n)])

def birth_distn_mutn(birth_distn, increment = 0.05):
    n = len(birth_distn)
    V = RealDistribution('uniform', [0, n])
    pm = U.get_random_element()
    #Decide if mutation adds or subtracts from parent birth_distn
    if pm < 1/2:
        plus_minus = +1
    else:
        plus_minus = -1
    #Decide the savings level probability to alter
    k = V.get_random_element()
    k = floor(k)
    new_birth_distn = list(birth_distn)
    new_val = birth_distn[k] + plus_minus*increment
    if new_val < 0:
        new_birth_distn[k] = 0
    elif new_val > 1:
        new_birth_distn[k] = 1
    else:
        new_birth_distn[k] = new_val
    new_birth_distn = tuple(new_birth_distn)
    return(new_birth_distn)

def death_process(squirrels, survival = [1/2, 1/2, 5/6, 5/6, 0]):
    dead_squirrels = []
    for s in squirrels:
        d = U.get_random_element()
        squirrel_dies = d > survival[s[1]]
        if squirrel_dies:
            dead_squirrels.append(s)
    for dead in dead_squirrels:
        squirrels.remove(dead)
    return(squirrels)

def birth_process(squirrels, mutation_probability = 0):
    baby_squirrels = []
    for i, s in enumerate(squirrels):
        b = U.get_random_element()
        gives_birth = b < s[0][s[1]]
        if gives_birth:
            m = U.get_random_element()
            mutates = m < mutation_probability
            if mutates:
                birth_distn = birth_distn_mutn(s[0])
            else:  #no mutation, inherits birth_distn of parent
                birth_distn = s[0]
            baby_squirrels.append(single_squirrel_maker(birth_distn))
        else:  #not giving birth
            squirrels[i] = single_squirrel_maker(s[0], s[1] + 1)
    squirrels = squirrels + baby_squirrels
    return(squirrels)

In [220]:
squirrels = many_squirrel_maker(100, (0, 0, 1))
len(death_process(squirrels))

43

In [195]:
squirrels = many_squirrel_maker(100, (1/2, 0, 1))
len(birth_process(squirrels, mutation_probability=1))

155

In [197]:
def single_day(squirrels, survival = [1/2, 1/2, 5/6]):
    squirrels = death_process(squirrels, survival = survival)
    squirrels = birth_process(squirrels)
    return(squirrels)

In [198]:
def many_days(n, squirrels, survival = [1/2, 1/2, 5/6]):
    for _ in range(n):
        squirrels = death_process(squirrels, survival = survival)
        squirrels = birth_process(squirrels)
    return(len(squirrels))

In [207]:
results = []
squirrels = many_squirrel_maker(100, (1/2, 0, 1))
for _ in range(100):
    x = many_days(100, squirrels)
    results.append(x)
mean(results).n()

32.2300000000000

In [208]:
results = []
squirrels = many_squirrel_maker(100, (0, 0, 1))
for _ in range(100):
    x = many_days(100, squirrels)
    results.append(x)
mean(results).n()

62.5200000000000

#### Everything up until now provides a reasonable sanity check; things work as they ought to.  Now let's instantiate an evolutionary process which walks us to the optimal birth_distn.

In [282]:
#Problem: Now that we allow squirrels' offspring to have different birth_distn, it is possible that a squirrel
#    not know with what probability to give birth; thus at the start of each day we'll have to run a check,
#    verifying that the squirrels know what to do. What makes most sense? A straightforward option is to insist
#    that the last entry be 1 in the birth_distn; if it isn't, add on another one, and make it 1. 
def check_squirrels(squirrels):
    for i, s in enumerate(squirrels):
        if s[0][-1] != 1:
            banked = s[1]
            birth_distn = list(s[0])
            birth_distn = birth_distn + [1]
            squirrels[i] = single_squirrel_maker(tuple(birth_distn), banked)
    return(squirrels)

#Another problem: a large population becomes hard to deal with. For that reason, I'll let the population grow to
#    size 1000, and then I'll kill of 90% of it uniformly at random (all we care about is frequency of
#    different types of squirrels, so that this is okay).
def cull_squirrels(squirrels, death_prob = 0.9):
    alive_squirrels = []
    for s in squirrels:
        v = U.get_random_element()
        squirrel_survives = v > death_prob
        if squirrel_survives:
            alive_squirrels.append(s)
    squirrels = alive_squirrels
    return(squirrels)

def hundred_to_hundred(squirrels, mutation_probability = 0.05, death_prob = 0.9):
    while len(squirrels) < 1000 and len(squirrels) > 0:
        check_squirrels(squirrels)
        squirrels = death_process(squirrels)
        squirrels = birth_process(squirrels, mutation_probability)
    squirrels = cull_squirrels(squirrels, death_prob)
    return(squirrels)

#Not a problem, just a tool: I want to understand what my population of squirrels looks like. 

def make_birth_freq_dict(squirrels):
    freq = {} 
    birth_distns = [tuple(s[0]) for s in squirrels]
    for s in birth_distns: 
        if (s in freq): 
            freq[s] += 1
        else: 
            freq[s] = 1
    return(freq)

In [336]:
squirrels = many_squirrel_maker(100, (0, 0, 0, 1))

In [337]:
for _ in range(100):
    if not _ % 10:
        print('running {}'.format(_))
    squirrels = hundred_to_hundred(squirrels)
make_birth_freq_dict(squirrels)

running 0
running 10
running 20
running 30
running 40
running 50
running 60
running 70
running 80
running 90


{}

In [320]:
squirrels

[[(6.93889390390723e-17, 0, 1, 1, 0.900000000000000, 1), 2],
 [(6.93889390390723e-17, 0, 0.950000000000000, 1, 0.950000000000000, 1), 3],
 [(0.100000000000000, 0, 0.950000000000000, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0, 1, 0.950000000000000, 0.900000000000000, 1), 2],
 [(6.93889390390723e-17, 0, 0.950000000000000, 1, 0.950000000000000, 1), 2],
 [(0.0500000000000001, 0, 1, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0.0500000000000000, 1, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0, 1, 0.950000000000000, 0.900000000000000, 1), 2],
 [(0.100000000000000, 0, 1, 0.950000000000000, 0.950000000000000, 1), 2],
 [(0.0500000000000001, 0, 1, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0, 0.950000000000000, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0, 0.950000000000000, 1, 0.900000000000000, 1), 2],
 [(6.93889390390723e-17, 0, 1, 1, 0.900000000000000, 1), 2],
 [(0.0500000000000001, 0.0500000000000000, 1, 1, 0.900000000000000, 1), 2],
 [(0.1